In [1]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}
</style>

In [2]:
from PIL import Image
from pathlib import Path
import json
from concurrent.futures import ThreadPoolExecutor
from itertools import repeat
import numpy as np
from tqdm.auto import tqdm
from shutil import rmtree 

SRC = Path.cwd()
DATA_PATH = Path.cwd().parent / "data" if Path.cwd().name == "src" else Path.cwd() / "data"

DATA_PATH, SRC

(PosixPath('/workspaces/ds340-project/data'),
 PosixPath('/workspaces/ds340-project/src'))

In [3]:
Path(DATA_PATH / "mod_input").mkdir(exist_ok=True)


def remake(dir):
    if dir.exists():
        rmtree(dir)
    dir.mkdir(parents=True)
    
for inner in ["train", "valid", "test"]:
    with tqdm(total=101, desc=str(inner)) as pbar, ThreadPoolExecutor() as executor:
        for _ in executor.map(remake, [Path(DATA_PATH / "mod_input" / inner / f"class_{label}") for label in range(0, 100+1)]):
            pbar.update(1)

train:   0%|          | 0/101 [00:00<?, ?it/s]

valid:   0%|          | 0/101 [00:00<?, ?it/s]

test:   0%|          | 0/101 [00:00<?, ?it/s]

In [4]:
Path(DATA_PATH / "mod_input").mkdir(exist_ok=True)

def fill_blanks(img_path):
    sink = str(img_path).replace("nobg", "mod")
    if Path(sink).exists(): return

    img = np.array(Image.open(img_path))
    
    non_zero = np.all((img != [0, 0, 0]), axis=-1) # all channels are non-zero
    color = np.sum(img[non_zero], axis=(0))
    replace_color = abs(np.round(color / np.sum(non_zero)) - 255)
    img[np.logical_not(non_zero)] = replace_color
    
    Image.fromarray(img).save(sink)


In [5]:
for source in ["train", "valid", "test"]:
    files = list(Path(DATA_PATH / "nobg_input" / source).rglob("*.*"))
    with tqdm(total=len(files), desc=source) as pbar, ThreadPoolExecutor() as executor:
        for _ in executor.map(fill_blanks, files):
            pbar.update(1)


train:   0%|          | 0/75732 [00:00<?, ?it/s]

/tmp/ipykernel_28042/3602867495.py:11: RuntimeWarning: invalid value encountered in divide
  replace_color = abs(np.round(color / np.sum(non_zero)) - 255)
/tmp/ipykernel_28042/3602867495.py:12: RuntimeWarning: invalid value encountered in cast
  img[np.logical_not(non_zero)] = replace_color


valid:   0%|          | 0/15150 [00:00<?, ?it/s]

test:   0%|          | 0/10100 [00:00<?, ?it/s]